<a href="https://colab.research.google.com/github/silverstar0727/pose-estimation/blob/main/continuous_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 해당 셀을 실행한 후에 반드시 "런타임 다시시작"을 해주세요
!pip install -q kfp
!pip3 install --user kfp google-cloud-aiplatform matplotlib --upgrade -q
# 추가
!pip3 install --user google-cloud-aiplatform --upgrade -q
!pip3 install --user kfp google-cloud-pipeline-components --upgrade -q

In [1]:
from datetime import datetime
import time

import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component, ClassificationMetrics, Metrics)
from kfp.v2.google.client import AIPlatformClient
from kfp.v2.google import experimental
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [2]:
from google.colab import auth as google_auth

google_auth.authenticate_user() # 사용할 gcp 계정으로 연결해주세요

In [13]:
PROJECT_ID = 'natural-expanse-319203'
REGION = "us-central1"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "gs://mediapipe-pipeline"

USER = "JeongMin-Do"
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/{USER}"

WORKING_DIR = PIPELINE_ROOT
MODEL_DISPLAY_NAME = f"train_deploy"
print(WORKING_DIR)
print(MODEL_DISPLAY_NAME)


aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_NAME,
    #experiment='my-experiment',
    #experiment_description='my experiment decsription'
)


gs://mediapipe-pipeline/pipeline_root/JeongMin-Do
train_deploy


In [27]:
@component(base_image="silverstar456/mediapipe:landmarks")
def get_landmarks(result_csv: OutputPath("result_csv")):
    import csv
    import cv2
    import numpy as np
    import os
    import sys
    import tqdm
    import pandas as pd

    from mediapipe.python.solutions import drawing_utils as mp_drawing
    from mediapipe.python.solutions import pose as mp_pose

    import wget 
    import zipfile

    # GCS에서 이미지 파일 다운로드 
    wget.download("https://storage.googleapis.com/mediapipe-pipeline/study.zip")

    # 압축해제
    fantasy_zip = zipfile.ZipFile('study.zip')
    fantasy_zip.extractall('.')
    fantasy_zip.close()

    def landmarks(input_frame):
        input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)

        # Initialize fresh pose tracker and run it.
        with mp_pose.Pose(upper_body_only=True) as pose_tracker:
            result = pose_tracker.process(image=input_frame)
            pose_landmarks = result.pose_landmarks
        
        # Save landmarks.
        if pose_landmarks is not None:
            # Check the number of landmarks and take pose landmarks.
            assert len(pose_landmarks.landmark) == 25, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
            pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

            # Map pose landmarks from [0, 1] range to absolute coordinates to get
            # correct aspect ratio.
            frame_height, frame_width = input_frame.shape[:2]
            pose_landmarks *= np.array([frame_width, frame_height, frame_width])

            # Write pose sample to CSV.
            pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.float64).tolist()

            return pose_landmarks

    images_in_folder = 'study'
    csv_out_path = result_csv

    with open(csv_out_path, 'w') as csv_out_file:
        csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)

        # Folder names are used as pose class names.
        pose_class_names = sorted([n for n in os.listdir(images_in_folder) if not n.startswith('.')])

        for pose_class_name in pose_class_names:
            print('Bootstrapping ', pose_class_name, file=sys.stderr)

            image_names = sorted([
                n for n in os.listdir(os.path.join(images_in_folder, pose_class_name))
                if not n.startswith('.')])
            for image_name in tqdm.tqdm(image_names, position=0):
                # Load image.
                input_frame = cv2.imread(os.path.join(images_in_folder, pose_class_name, image_name))
                pose_landmarks = landmarks(input_frame)

                try:
                    csv_out_writer.writerow([image_name, pose_class_name] + pose_landmarks)
                except:
                    pass       

In [28]:
# silverstar456/mediapipe:knn
# us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest
@component(base_image="silverstar456/mediapipe:knn")
def knn_model(result_csv: InputPath("result_csv"), 
        model_output: OutputPath("model")
        ):
    import pandas as pd
    import os
    from google.cloud import storage

    import pickle
    from joblib import dump, load
    from sklearn.neighbors import KNeighborsClassifier

    data = pd.read_csv(result_csv, header=None)
    x_train, y_train = data.iloc[:, 2:], data.iloc[:, 1]

    classifier = KNeighborsClassifier(n_neighbors = 3)

    classifier.fit(x_train, y_train)

    dump(classifier, model_output) 

    #blob = storage.blob.Blob.from_string(OutputPath, client=storage.Client())
    #blob.upload_from_filename('model.joblib')

    # clf = load('filename.joblib') 

In [29]:
@component(base_image="silverstar456/mediapipe:test")
def model_test(result_csv: InputPath("result_csv"), model_output: InputPath("model")):
    import cv2
    import numpy as np
    import os
    import sys
    import tqdm
    import pandas as pd
    from joblib import dump, load

    from sklearn.neighbors import KNeighborsClassifier

    from mediapipe.python.solutions import drawing_utils as mp_drawing
    from mediapipe.python.solutions import pose as mp_pose

    def landmarks(input_frame):
        input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)

        # Initialize fresh pose tracker and run it.
        with mp_pose.Pose(upper_body_only=True) as pose_tracker:
            result = pose_tracker.process(image=input_frame)
            pose_landmarks = result.pose_landmarks
        
        # Save landmarks.
        if pose_landmarks is not None:
            # Check the number of landmarks and take pose landmarks.
            assert len(pose_landmarks.landmark) == 25, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
            pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

            # Map pose landmarks from [0, 1] range to absolute coordinates to get
            # correct aspect ratio.
            frame_height, frame_width = input_frame.shape[:2]
            pose_landmarks *= np.array([frame_width, frame_height, frame_width])

            # Write pose sample to CSV.
            pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.float64).tolist()

            return pose_landmarks

    classifier = load(model_output) 

    data = pd.read_csv(result_csv, header=None)
    x, y = data.iloc[:, 2:], data.iloc[:, 1]
    correct = 0
    for i in range(len(data)):
        sample = np.array(x.iloc[i, :]).reshape(1, -1)
        pred = classifier.predict(sample)
        if pred[0] == y[i]:
            correct += 1
            
    print(correct / len(data))

In [30]:
@dsl.pipeline(
    name = "mediapipe-pipeline2",
    description = "mediapipe",
    pipeline_root=PIPELINE_ROOT
)
def mediapipe():
    landmarks = get_landmarks()
    model = knn_model(landmarks.output)
    test = model_test(landmarks.output, model.output)



"""
    model_upload_op = aiplatform.Model.upload(
        display_name=MODEL_DISPLAY_NAME,
        serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest")
    
    model_upload_op.after(model)"""


compiler.Compiler().compile(
    pipeline_func = mediapipe, 
    package_path = "mediapipe.json"
)

api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

response = api_client.create_run_from_job_spec(
    job_spec_path="mediapipe.json"
)

UnboundLocalError: ignored